In [ ]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
from IPython.display import Audio
import ipywidgets as widgets
import folium

# Conectarse a la base de datos

In [ ]:
MONGO_URI = 'mongodb://localhost:27017'
client = MongoClient(MONGO_URI)

db = client['FuSA_Diego']
collectionArchivos = db["archivos"]

## generar los datos necesarios para los widgets

In [ ]:
## Fechas sin repeticion
setFechas = []
fechasSubidas = collectionArchivos.find({},{'fecha_grabacion':1,'_id':0})
for i in fechasSubidas:
    setFechas.append(i.get("fecha_upload"))
setFechas.append("Todo")
setFechas = set(setFechas)

elegirFechas = widgets.Combobox(
                                placeholder='DD-MM-YY/(Todo)',
                                options=list(setFechas),
                                description='Fecha:',
                                ensure_option=True,
                                disabled=False
                                )

## Categorias
categorias = ['Humanos','Musica','Animales','Medio ambientales','Mecanicos','Vehiculos','Alertas']
def crearCheckBox(categoria):
    return widgets.Checkbox(
                            value=False,
                            description=categoria,
                            disabled=False
                            )
listaCB = [crearCheckBox(i) for i in categorias]

## Crear mapa

In [ ]:
m = folium.Map(location=[-39.814567654937946,-73.24711789824227],zoom_start=14)
def boton_apretado(b):
    categoriasElegidas = []
    for tipo in listaCB:
        if (tipo.value):
            categoriasElegidas.append(tipo.description)
    if (elegirFechas.value!="Todo"):
        fechaElegida = elegirFechas.values
    else:
        fechaElegida = ""
    
    condicion = {"fuentes.tipo_fuente":{'$in':categoriasElegidas},'fecha_grabacion':{'$regex':fechaElegida+"$"}}
    mostrar = {'latitud':1,'longitud':1,'_id':0,"fuentes.descripcion":1,'audio':1}
    consulta = collectionArchivos.find(condicion,mostrar)
    for j,i in enumerate(consulta):
        crearMarcadores(i.get('latitud'),i.get('longitud'),i.get('fuentes')[0].get('descripcion'),f"Audio {j+1}")
        print(f"Audio {j+1}:")
        display(Audio(i.get('audio')))
        
    coordenadas = collectionArchivos.find({},{'latitud':1,'longitud':1,'_id':0}) ## cambiar    
    imprimirMapa()
#---------------------------------------------

def crearMarcadores(latitud,longitud,descripcion,tooltip):
    frase = "<i>"+descripcion+"</i>"
    m.add_child(folium.Marker(
                [latitud,longitud], popup=frase,tooltip=tooltip  
                ))
    
def imprimirMapa():
    display(m)

coordenadas = collectionArchivos.find({},{'latitud':1,'longitud':1,'_id':0}) ## cambiar
push_data = widgets.Button(description='Aplicar cambios')
display(elegirFechas)
display(widgets.Box(listaCB))
display(push_data)
push_data.on_click(boton_apretado)